In [ ]:
!pip install gurobipy
!pip install cplex
!pip install docplex
!pip install ortools

###Data

In [ ]:
from docplex.cp.model import CpoModel
import cplex
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from google.colab import drive
drive.mount('/content/drive')
import time

def read_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

        Teams = list(map(int, lines[0].strip().split()))
        T = int(lines[1].strip())  # Number of teams
        W = int(lines[2].strip())  # Number of weeks
        c = list(map(int, lines[3].strip().split()))  # Conferences
        Conferences = int(lines[4].strip()) # Number of conferences

        # Read distance matrix (from line 5 onwards)
        d = []
        for line in lines[5:]:
            row = list(map(int, line.strip().split()))
            d.append(row)

        return Teams, T, W, c, Conferences, d
#-------------------------------  import data ----------------------------------------
# Start the timer
start_time = time.time()

# Call the read_file function to load data
datafile = "/content/drive/MyDrive/MIE562 Project/Smaller Instances/inst10.txt"
Teams, num_teams, W, C, Conferences, distances = read_file(datafile)

distances = np.array(distances) # Convert to NumPy array

print("Distance Matrix:")
print(distances)

#-------------------------------  build the model ----------------------------------------

# Define the constant sets
T = list(range(num_teams))  # Teams {0, 1, ..., num_teams - 1}
'''
C = df.iloc[3, :].tolist()  # Conferences
C.extend([np.nan] * (num_teams - len(C)))
W = range(11)  # Weeks {0, 1, ..., 10}
D = range(2)   # Days {0, 1}
G = 22        # Number of games per team
'''
# Create the model
model = cp_model.CpModel()

# Decision variables
xi = [[[model.NewIntVar(0, 1, f'xi_{i}_{j}_{k}') for k in range(2)] for j in T] for i in T]
ti = [[model.NewIntVar(0, 1, f'ti_{i}_{j}') for j in T] for i in T]
Di = [model.NewIntVar(0, int(sum(distances[i])), f'Di_{i}') for i in T]
Dmax = model.NewIntVar(0, int(sum([sum(row) for row in distances])), 'Dmax')


# Total travel distance calculation
for i in T:
    model.Add(Di[i] == sum(distances[i][j] * (xi[i][j][0] + xi[j][i][1]) for j in T if i != j))

# Minimize total travel distance
#model.Minimize(sum(Di[i] for i in T))

# Each team plays exactly (num_teams - 1) games
for i in T:
    model.Add(sum(xi[i][j][0] + xi[i][j][1] for j in T if i != j) == num_teams - 1)

# Dmax is at least the maximum distance for any team
for i in T:
    model.Add(Dmax >= Di[i])

diff = {}
# Equal home and away games (or differ by 1 for odd number of teams)
for i in T:
    diff[i] = model.NewIntVar(-num_teams, num_teams, f'diff_{i}')  # Creates integer variable diff i to represent the absolute differnece between the number of home and away games
    model.AddAbsEquality(diff[i], sum(xi[i][j][0] for j in T if i != j) - sum(xi[i][j][1] for j in T if i != j))
    model.Add(diff[i] <= 1)  # Constrains the absolute difference


# Each pair of teams plays exactly one match
for i in T:
    for j in T:
        if i != j:
            model.Add(xi[i][j][0] + xi[i][j][1] == 1)

# Team can't play against itself
for i in T:
    model.Add(xi[i][i][0] == 0)  # No home game against itself
    model.Add(xi[i][i][1] == 0)  # No away game against itself

# Objective function: Minimize the maximum total travel distance
total_distance = sum(distances[i][j] * (xi[i][j][0] + xi[i][j][1]) for i in T for j in T if i != j)
model.Minimize(total_distance)

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

#-------------------------------  display the results ----------------------------------------
# Display results

if status == cp_model.OPTIMAL:
    print("Optimal solution found.")
    print(f"Total distance traveled: {solver.ObjectiveValue()}")
    # Print total distance for each team
    for i in T:
        team_total_distance = sum(distances[i][j] * (solver.Value(xi[i][j][0]) + solver.Value(xi[i][j][1])) for j in T if i != j)
        print(f"Total distance traveled by team {i+1}: {team_total_distance}")
    for i in T:
        print(f'Total travel distance for team {i+1}: {solver.Value(Di[i])}')
        for j in T:
            if i != j:
                if solver.Value(xi[i][j][0]) == 1:
                    print(f'Team {i+1} plays at home against team {j+1}')
                if solver.Value(xi[i][j][1]) == 1:
                    print(f'Team {i+1} plays away against team {j+1}')
elif status == cp_model.FEASIBLE:
    print("Feasible solution found.")
    total_distance_minimized = sum(solver.Value(Di[i]) for i in T)
    print(f"Total distance minimized: {total_distance_minimized}")
    for i in T:
        print(f'Total travel distance for team {i+1}: {solver.Value(Di[i])}')
        for j in T:
            if i != j:
                if solver.Value(xi[i][j][0]) == 1:
                    print(f'Team {i+1} plays at home against team {j+1}')
                if solver.Value(xi[i][j][1]) == 1:
                    print(f'Team {i+1} plays away against team {j+1}')
else:
    print("No feasible solution found.")

# End the timer
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")

